In [ ]:
import numpy
import cv2
import matplotlib.pyplot as plot


In [ ]:
def HarrisDetector(src: numpy.ndarray, blockSize: int, apertureSize: int,
                    k: float, threshold: float, disjointDiameter: int) -> list:

    # src should be BGR or CV_8UC3 image 
    # apertureSize should > 0

    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

    # according to cornerEigenValsVecs() in OpenCV imgproc corner.cpp
    scale = 1.0 / ((1 << (apertureSize - 1)) * blockSize * 255)
    Dx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, None, apertureSize, scale)
    Dy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, None, apertureSize, scale)

    DxDx = Dx**2
    DxDy = Dx*Dy
    DyDy = Dy**2

    # according to cornerEigenValsVecs() in OpenCV imgproc corner.cpp
    # cv2.GaussianBlur(DxDx, (blockSize, blockSize), sigma, DxDx)
    # cv2.GaussianBlur(DxDy, (blockSize, blockSize), sigma, DxDy)
    # cv2.GaussianBlur(DyDy, (blockSize, blockSize), sigma, DyDy)
    cv2.boxFilter(DxDx, cv2.CV_32F, (blockSize, blockSize), DxDx, normalize=False)
    cv2.boxFilter(DxDy, cv2.CV_32F, (blockSize, blockSize), DxDy, normalize=False)
    cv2.boxFilter(DyDy, cv2.CV_32F, (blockSize, blockSize), DyDy, normalize=False)

    HarrisResponse = DxDx*DyDy - DxDy**2 - k*(DxDx + DyDy)**2

    HarrisThreshold = HarrisResponse > numpy.amax(HarrisResponse) * threshold

    # non-maximum suppression
    HarrisResponseMaximum = HarrisResponse == cv2.dilate(HarrisResponse,
            cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (disjointDiameter, disjointDiameter)))

    keypointsIndex = numpy.nonzero(HarrisThreshold & HarrisResponseMaximum)

    # apply a blur filter for keypoint angle calculation. sigma?
    cv2.GaussianBlur(Dx, (apertureSize, apertureSize), 4/apertureSize, Dx)
    cv2.GaussianBlur(Dy, (apertureSize, apertureSize), 4/apertureSize, Dy)

    # angle is in clockwise in OpenCV KeyPoint class
    # angle is in counter-clockwise in NumPy arctan2()
    keypoints = [cv2.KeyPoint(numpy.float32(x), numpy.float32(y), disjointDiameter,
                                numpy.degrees(-numpy.arctan2(Dy[y, x], Dx[y, x])),
                                HarrisResponse[y, x])
                    for y, x in numpy.transpose(keypointsIndex)]

    list.sort(keypoints, key=lambda keypoint: keypoint.response, reverse=True)

    return keypoints



In [ ]:
def SimpleDescriptor(src: numpy.ndarray, keypoints: list, sampleSize: int) -> numpy.ndarray:

    # src should be BGR or CV_8UC3 image

    src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY).astype(numpy.float32)

    src = numpy.pad(src, ((sampleSize//2, sampleSize-1-sampleSize//2), 
                            (sampleSize//2, sampleSize-1-sampleSize//2)), "reflect")

    # ndarray with ndim 2 and shape (len(keypoints), sampleSize**2)
    descriptors = numpy.empty((len(keypoints), sampleSize**2), dtype=src.dtype)

    for index, keypoint in enumerate(keypoints):
        x = int(keypoint.pt[0])
        y = int(keypoint.pt[1])
        descriptors[index] = src[y:y+sampleSize, x:x+sampleSize].reshape(-1)

    return descriptors



In [ ]:
def MOPSDescriptor(src: numpy.ndarray, keypoints: list,
                    sampleSize: int, spaceSize: int) -> numpy.ndarray:

    # src should be BGR or CV_8UC3 image

    src = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY).astype(numpy.float32)

    # times 1.5 for rotate k*45 degrees
    padding = int(sampleSize/2 * spaceSize*1.5)

    src = numpy.pad(src, ((padding, padding), (padding, padding)), mode="reflect")

    # ndarray with ndim 2 and shape (len(keypoints), sampleSize**2)
    descriptors = numpy.empty((len(keypoints), sampleSize**2), dtype=src.dtype)

    slicingStart = padding-sampleSize//2
    slicingStop  = slicingStart+sampleSize

    for index, keypoint in enumerate(keypoints):
        x = int(keypoint.pt[0])
        y = int(keypoint.pt[1])

        img = src[y:y+padding*2, x:x+padding*2]

        # img = cv2.GaussianBlur(img, ksize, sigma)

        img = cv2.warpAffine(img,
                cv2.getRotationMatrix2D((padding, padding), keypoint.angle, 1/spaceSize), None)

        descriptors[index] = img[slicingStart: slicingStop, slicingStart: slicingStop].reshape(-1)
        descriptors[index] -= numpy.mean(descriptors[index])
        descriptors[index] /= numpy.std(descriptors[index])

    return descriptors



In [ ]:
def DifferenceMatcher(descriptors1_: numpy.ndarray, descriptors2_: numpy.ndarray,
                        order: int = 2, method: str = "difference") -> list:

    # exist a bug that a keypoint may be matched with other keypoints for several times
    # descriptors should be a ndarray with ndim 2 and shape (len(keypoints), sampleSize**2)

    # the difference array will have a shape (m, n, s**2) where m >= n
    if descriptors1_.shape[0] < descriptors2_.shape[0]:
        swap = True
        descriptors1 = numpy.expand_dims(descriptors2_, axis=1)
        descriptors2 = numpy.expand_dims(descriptors1_, axis=0)
    else:
        swap = False
        descriptors1 = numpy.expand_dims(descriptors1_, axis=1)
        descriptors2 = numpy.expand_dims(descriptors2_, axis=0)

    # order = 2: sum of   square differences (SSD)
    if order == 2:
        difference = numpy.sum((descriptors1 - descriptors2) ** order, axis=-1)
    # order = 1: sum of absolute differences (SAD)
    elif order == 1:
        difference = numpy.sum(numpy.absolute(descriptors1 - descriptors2), axis=-1)
    else:
        raise ValueError

    # 用较少描述子的数组匹配较多描述子的数组，按各对描述子的差值排序从小到大排序差值
    differenceSorted = numpy.sort(difference, axis=0)
    # 按各对描述子的差值排序从小到大排序索引
    differenceSortedIndex = numpy.argsort(difference, axis=0)

    if method == "difference":
        if swap:
            matches = [cv2.DMatch(index, differenceSortedIndex[0, index],
                                    differenceSorted[0, index])
                        for index in range(descriptors2.shape[1])]
        else:
            matches = [cv2.DMatch(differenceSortedIndex[0, index], index,
                                    differenceSorted[0, index])
                        for index in range(descriptors2.shape[1])]

    elif method == "ratio":
        if swap:
            matches = [cv2.DMatch(index, differenceSortedIndex[0, index], 
                                    differenceSorted[0, index] / differenceSorted[1, index])
                        for index in range(descriptors2.shape[1])]
        else:
            matches = [cv2.DMatch(differenceSortedIndex[0, index], index,
                                    differenceSorted[0, index] / differenceSorted[1, index])
                        for index in range(descriptors2.shape[1])]
    else:
        raise ValueError

    list.sort(matches, key=lambda match: match.distance)

    return matches



In [ ]:
def EvaluateMatcher(keypoints1: list, keypoints2: list, matches: list,
                    transform: numpy.ndarray, threshold: float = 1.0):

    # transform should be a ndarray with shape (3, 3)

    TruePositiveCounter = 0
    FalsPositiveCounter = 0
    TruePositiveList = numpy.empty(len(matches), dtype=int)
    FalsPositiveList = numpy.empty(len(matches), dtype=int)

    # let length be len(matches)
    #  total positive amount is TruePositiveList[length]
    #  total negative amount is FalsPositiveList[length]

    # for matches before an arbitrary index:
    #  true  positive amount is TruePositiveCounter
    #  false positive amount is FalsPositiveCounter
    #  true  negative amount is FalsPositiveList[length] - FalsPositiveCounter
    #  false negative amount is TruePositiveList[length] - TruePositiveCounter

    # so
    # * True  Positive Rate (TPR) /   Sensitivity =   (true  positive)  / (total positive)
    #                             /        Recall = TruePositiveCounter / TruePositiveList[length]
    #   True  Negative Rate (TNR) /   Specificity =   (true  negative)  / (total negative)
    # * False Positive Rate (FPR) / 1-Specificity =   (true  positive)  / (total negative)
    #                                             = FalsPositiveCounter / FalsPositiveList[length]

    # Receiver Operating Characteristic (ROC) is defined as Sensitivity (TPR) against 1-Specificity (FPR)

    for index, match in enumerate(matches):

        # (3) = (3, 3) @ (3) | (3) <- (2) + (1)
        mappedPoint = transform @ numpy.append(keypoints1[match.queryIdx].pt, 1)
        # (x", y", 1) <- (x', y', w')
        mappedPoint /= mappedPoint[2]

        if numpy.all(numpy.absolute(keypoints2[match.trainIdx].pt - mappedPoint[0:2]) < threshold):
            TruePositiveCounter += 1
        else:
            FalsPositiveCounter += 1

        TruePositiveList[index] = TruePositiveCounter
        FalsPositiveList[index] = FalsPositiveCounter

    # Area Under Curve (AUC) is defined as the area between the polyline and the x-axis

    # to calculate the area, divide x-axis in n piece,
    # each piece is a rectangle with height y (n and y will discuss later),
    # so AUC is the sum of area of these n rectangle

    # the length of x-axis can be regarded as FalsPositiveCounter, with "length" points on x-axis
    # some of these points will overlap,
    # so the length dx between a pair of point can be calcuated by "the next point" subtract "this point"

    # note that y can be regarded as FalsPositiveList[index],
    # so the area of each piece is dx * FalsPositiveList[index]

    aucFalsPositive = FalsPositiveList - numpy.concatenate((FalsPositiveList[0].reshape(1), FalsPositiveList[:-1]))
    auc = numpy.sum(aucFalsPositive * TruePositiveList) / FalsPositiveCounter / TruePositiveCounter

    # TPR array, FPR array, AUC, Positive amount, Negative amount
    return TruePositiveList/TruePositiveCounter, FalsPositiveList/FalsPositiveCounter,\
            auc, TruePositiveCounter, FalsPositiveCounter



In [ ]:
def main(src1, src2, transformMatrix,
    # Detector
        # Harris Detector
        HarrisDetectorBlockSize=2, HarrisDetectorApertureSize=3, HarrisDetectorK=0.04,
        HarrisDetectorThreshold=0.01, HarrisDetectorDisjointDiameter=10,
    # Descriptor
    Descriptor="MOPS",
        # Simple Descriptor
        SimpleDescriptorSize=5,
        # MOPS Descriptor
        MOPSDescriptorSample=8, MOPSDescriptorSpace=5,
    # Matcher
    Matcher="Difference",
        # Difference Matcher
        DifferenceMatcherMethod="ratio",
        # OpenCV Brute-Force Matcher
        BFMatcherCrossCheck=False,
    # display
    displayMatches=True, displayMatchesPercentage=25
):

    # Detector
    keypoints1 = HarrisDetector(src1, HarrisDetectorBlockSize, HarrisDetectorApertureSize,
                    HarrisDetectorK, HarrisDetectorThreshold, HarrisDetectorDisjointDiameter)
    keypoints2 = HarrisDetector(src2, HarrisDetectorBlockSize, HarrisDetectorApertureSize,
                    HarrisDetectorK, HarrisDetectorThreshold, HarrisDetectorDisjointDiameter)

    # Descriptor
    if Descriptor == "Simple":
        descriptor1 = SimpleDescriptor(src1, keypoints1, SimpleDescriptorSize)
        descriptor2 = SimpleDescriptor(src2, keypoints2, SimpleDescriptorSize)
        sDescriptor = f"Descriptor: Simple, {SimpleDescriptorSize}\n"
    elif Descriptor == "MOPS":
        descriptor1 = MOPSDescriptor(src1, keypoints1, MOPSDescriptorSample, MOPSDescriptorSpace)
        descriptor2 = MOPSDescriptor(src2, keypoints2, MOPSDescriptorSample, MOPSDescriptorSpace)
        sDescriptor = f"Descriptor: MOPS, {MOPSDescriptorSample}, {MOPSDescriptorSpace}\n"
    else:
        raise ValueError

    # Matcher
    if Matcher == "Difference":
        matches = DifferenceMatcher(descriptor1, descriptor2, method=DifferenceMatcherMethod)
        sMatcher = f"Matcher: Difference, {DifferenceMatcherMethod}\n"
    elif Matcher == "BF":
        BFMatcher = cv2.BFMatcher_create(crossCheck=BFMatcherCrossCheck)
        matches = sorted(BFMatcher.match(descriptor1, descriptor2), key=lambda match: match.distance)
        sMatcher = f"Matcher: OpenCV Brute-Force\n"
    else:
        raise ValueError

    # Evaluate Matcher
    # True Positive Rate, False Positive Rate, Area Under Curve
    tpr, fpr, auc, correctMatch, incorrectMatch = EvaluateMatcher(keypoints1, keypoints2, matches, transformMatrix)
    sEvaluator = f"Match amount: {len(matches)}, T: {correctMatch}, F: {incorrectMatch}"

    # display image
    if displayMatches:
        # dst1 = cv2.drawKeypoints(src1, keypoints1, None, (255, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # dst2 = cv2.drawKeypoints(src2, keypoints2, None, (255, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # cv2.imshow(f"Harris Detector", numpy.concatenate(((dst1, dst2)), axis=1))
        dst = cv2.drawMatches(src1, keypoints1, src2, keypoints2, matches[:len(matches) * displayMatchesPercentage // 100],
                                None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv2.imshow(f"{sMatcher} | {sEvaluator}", dst)

    # draw ROC
    figure, axes = plot.subplots()
    axes.plot(fpr, tpr)
    axes.axline((0, 0), (1, 1), linewidth=0.5, color='g')
    axes.grid(True)
    axes.set(xticks=numpy.linspace(0.0, 1.0, 11), yticks=numpy.linspace(0.0, 1.0, 11),
                title="Receiver Operating Characteristic (ROC)",
                xlabel="1-Specificity / FPR", ylabel="Sensitivity / TPR")
    axes.text(0.45, 0.05, f"AUC: {round(auc*100, 2)}%\n" + sDescriptor + sMatcher + sEvaluator,
                backgroundcolor='w', linespacing=1.8)
    plot.show()

    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [ ]:
main(
    src1 = cv2.imread("res\\Yosemite1.jpg"),
    src2 = cv2.imread("res\\Yosemite2.jpg"),
    transformMatrix = numpy.loadtxt("res\\Yosemite1to2Matrix.txt"),

    # Detector
        HarrisDetectorBlockSize=2,
        HarrisDetectorApertureSize=3,
        HarrisDetectorK=0.04,
        HarrisDetectorThreshold=0.01,
        HarrisDetectorDisjointDiameter=10,

    # Descriptor
    # Descriptor="Simple",
    #     SimpleDescriptorSize=5,
    Descriptor="MOPS",
        MOPSDescriptorSample=8,
        MOPSDescriptorSpace=5,

    # Matcher
    Matcher="Difference",
        # DifferenceMatcherMethod="difference",
        DifferenceMatcherMethod="ratio",
    # Matcher="BF",
        # BFMatcherCrossCheck=True,

    # displayMatches=False,
        displayMatchesPercentage=10,
)
